# Asteroid use case notebook

ALeRCE starter notebook for asteroid science.

You will need to install psycopg2 and the alerce api

In this notebook you will connect to the ALeRCE database, and download asteroid candidates from the last 10 days

You will plot their spatial distribution using datashader and bokeh

### Load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import coordinates
from astropy.time import Time
from datetime import datetime
from IPython.display import HTML
from ipywidgets import Layout, Box, widgets

ALeRCE client

In [2]:
#!pip install alerce
from alerce.api import AlerceAPI
client = AlerceAPI()

Install psycopg2 https://github.com/psycopg/psycopg2

In [3]:
#!pip install psycopg2-binary
import psycopg2

### Get credentials (in github repository)

In [4]:
import json
credentials_file = "../alercereaduser_v2.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

### Connect to DB

In [5]:
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

### Show all the available tables

In [6]:
query = "select tablename from pg_tables where schemaname='public';"
tables = pd.read_sql_query(query, conn)
tables.sort_values(by="tablename")

,tablename
9,asassn
18,class
10,crtsnorth
11,crtssouth
3,detections
16,features
22,features_v2
2,features_v3
23,ingestion_timestamp
1,insert_tmp


### For each table, show column names and column types

In [7]:
alltabs = []
for tab in sorted(tables.tablename):
    cols = pd.DataFrame()
    query = "select column_name, data_type from information_schema.columns where table_name = '%s';" % tab
    results = pd.read_sql_query(query, conn)
    results["table"] = tab
    alltabs.append(results)
dftab = pd.concat(alltabs)
pd.options.display.max_rows = 999
display(dftab[["table", "column_name", "data_type"]])
pd.options.display.max_rows = 101

,table,column_name,data_type
0,asassn,ASAS-SN Name,text
1,asassn,Other Names,text
2,asassn,LCID,integer
3,asassn,ra,double precision
4,asassn,dec,double precision
5,asassn,Mean VMag,double precision
6,asassn,Amplitude,double precision
7,asassn,Period,double precision
8,asassn,Type,text
9,asassn,Url,text


### Query class numbers

In [8]:
query = "select * from class"
classes = pd.read_sql_query(query, conn)
classmapper = dict(zip(classes.name.tolist(), classes.id.tolist()))
classmapper

{'Other': 0,
 'Ceph': 1,
 'LPV': 4,
 'RRL': 5,
 'EB': 3,
 'SNe': 6,
 'DSCT': 2,
 'Blazar': 8,
 'CV/Nova': 9,
 'SLSN': 14,
 'AGN': 18,
 'SN': 19,
 'Variable Star': 20,
 'Asteroid': 21,
 'Bogus': 22,
 'SNIa': 10,
 'SNIbc': 11,
 'SNII': 12,
 'SNIIn': 13,
 'EBSD/D': 15,
 'EBC': 16,
 'Periodic-Other': 17,
 'AGN-I': 7,
 'RS-CVn': 23,
 'QSO-I': 24}

### Query objects which appeared during the last 10 days, which have at most one detection, and which are classified as asteroids by the early classifier with a probability greater than 0.95

Please contact us for larger queries. This query of only 10 days takes about 5 seconds.

In [13]:
mjd_last = Time(datetime.utcnow(), scale='utc').mjd - 10.

query='''
select objects.oid, objects.meanra, objects.meandec, objects.firstmjd,
objects.nobs, objects.classearly, objects.pclassearly

from objects

where objects.firstMJD > %s  and objects.nobs=1
and objects.classearly=%i and objects.pclassearly>0.95
''' % (mjd_last, classmapper["Asteroid"])

ast = pd.read_sql_query(query, conn)
ast.set_index('oid', inplace=True)
ast.head()

,meanra,meandec,firstmjd,nobs,classearly,pclassearly
oid,,,,,,
ZTF19actuhay,42.858072,33.068998,58806.242940,1,21,1.000000
ZTF19actumvz,0.785712,62.817990,58806.235382,1,21,0.999942
ZTF19actunes,40.024431,6.002280,58806.246713,1,21,1.000000
ZTF19actunjp,40.086531,10.604741,58806.246262,1,21,1.000000
ZTF19acturhh,32.830520,7.547021,58806.248102,1,21,1.000000


In [14]:
ast.shape

(138605, 6)

# Plot the stamps of 10 random candidates

In [15]:
for oid in np.random.choice(ast.index, 10):
    client.plot_stamp(oid)

# Plot distribution in the sky using datashader

In [16]:
import datashader as ds
import datashader.transfer_functions as tf
import bokeh.plotting as bp
from datashader.bokeh_ext import InteractiveImage

In [17]:
bp.output_notebook()
p = bp.figure(tools='pan,wheel_zoom,reset', x_range=(0,360), y_range=(-90,90), plot_width=800, plot_height=500)

def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(ast, 'meanra', 'meandec', ds.count())
    img = tf.shade(agg)
    return tf.dynspread(img)

InteractiveImage(p, image_callback)

Loading BokehJS ...